In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [2]:
#Create a spark session
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

23/02/18 22:42:37 WARN Utils: Your hostname, lenovo resolves to a loopback address: 127.0.1.1; using 192.168.1.10 instead (on interface wlp0s20f3)
23/02/18 22:42:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/18 22:42:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option("header", "true").option("inferSchema", "true")\
    .csv("/home/sakabuana31/DigitalSkola/Learning/latihan_spark/input/yellow_tripdata_2021-01.csv.gz")
print("Here is our inferred schema:")
df.printSchema()

Here is our inferred schema:
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [4]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2021-01-01 00:30:10|  2021-01-01 00:36:12|              1|          2.1|         1|                 N|         142|          43|           2|        8.0|  3.0|    0.5|       0.0|         0.0|                  0.3

In [5]:
df.createOrReplaceTempView("tripData")

In [12]:
sqlDF = spark.sql("SELECT DISTINCT VendorID,  tpep_pickup_datetime, tpep_dropoff_datetime FROM \
tripData where PULocationID is not null and DOLocationID ='151' and VendorID= '1' \
GROUP BY VendorID,  tpep_pickup_datetime, tpep_dropoff_datetime ORDER BY tpep_pickup_datetime ASC")
sqlDF.show()

+--------+--------------------+---------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|
+--------+--------------------+---------------------+
|       1| 2021-01-01 00:25:30|  2021-01-01 00:35:34|
|       1| 2021-01-01 00:51:20|  2021-01-01 00:52:19|
|       1| 2021-01-01 00:53:01|  2021-01-01 00:59:03|
|       1| 2021-01-01 00:56:57|  2021-01-01 01:04:21|
|       1| 2021-01-01 01:25:53|  2021-01-01 01:43:54|
|       1| 2021-01-01 01:26:07|  2021-01-01 01:38:13|
|       1| 2021-01-01 01:26:32|  2021-01-01 01:32:15|
|       1| 2021-01-01 01:38:54|  2021-01-01 01:57:07|
|       1| 2021-01-01 01:40:22|  2021-01-01 02:06:31|
|       1| 2021-01-01 01:43:26|  2021-01-01 01:53:31|
|       1| 2021-01-01 01:46:47|  2021-01-01 01:58:55|
|       1| 2021-01-01 01:46:49|  2021-01-01 02:05:22|
|       1| 2021-01-01 01:53:59|  2021-01-01 01:54:03|
|       1| 2021-01-01 01:56:17|  2021-01-01 02:15:17|
|       1| 2021-01-01 02:27:04|  2021-01-01 02:35:33|
|       1| 2021-01-01 02:33:

In [14]:
df_distinct = spark.sql("SELECT DISTINCT store_and_fwd_flag from tripData where PULocationID IS NOT NULL and DOLocationID is NOT NULL ")
df_distinct.show()

+------------------+
|store_and_fwd_flag|
+------------------+
|              null|
|                 Y|
|                 N|
+------------------+



In [15]:
sqlDF.write.parquet("/home/sakabuana31/DigitalSkola/Learning/latihan_spark/output/spark_write_parquet.parquet")

In [16]:
df_parquet = spark.read.parquet("/home/sakabuana31/DigitalSkola/Learning/latihan_spark/output/spark_write_parquet.parquet")
df_parquet.show()

+--------+--------------------+---------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|
+--------+--------------------+---------------------+
|       1| 2021-01-01 00:25:30|  2021-01-01 00:35:34|
|       1| 2021-01-01 00:51:20|  2021-01-01 00:52:19|
|       1| 2021-01-01 00:53:01|  2021-01-01 00:59:03|
|       1| 2021-01-01 00:56:57|  2021-01-01 01:04:21|
|       1| 2021-01-01 01:25:53|  2021-01-01 01:43:54|
|       1| 2021-01-01 01:26:07|  2021-01-01 01:38:13|
|       1| 2021-01-01 01:26:32|  2021-01-01 01:32:15|
|       1| 2021-01-01 01:38:54|  2021-01-01 01:57:07|
|       1| 2021-01-01 01:40:22|  2021-01-01 02:06:31|
|       1| 2021-01-01 01:43:26|  2021-01-01 01:53:31|
|       1| 2021-01-01 01:46:47|  2021-01-01 01:58:55|
|       1| 2021-01-01 01:46:49|  2021-01-01 02:05:22|
|       1| 2021-01-01 01:53:59|  2021-01-01 01:54:03|
|       1| 2021-01-01 01:56:17|  2021-01-01 02:15:17|
|       1| 2021-01-01 02:27:04|  2021-01-01 02:35:33|
|       1| 2021-01-01 02:33:

In [22]:
sqlTripData = spark.sql("SELECT VendorID, PUlocationID, DOlocationID, RatecodeID \
FROM tripData WHERE VendorID IS NOT NULL AND PULocationID IS NOT NULL and DOLocationID is NOT NULL AND tpep_pickup_datetime >= ' 2021-01-01' \
AND tpep_pickup_datetime <=' 2021-01-03' GROUP BY VendorID, PUlocationID, DOlocationID, RatecodeID \
ORDER BY VendorID")
sqlTripData.show()

+--------+------------+------------+----------+
|VendorID|PUlocationID|DOlocationID|RatecodeID|
+--------+------------+------------+----------+
|       1|         138|          96|         1|
|       1|         132|         220|         1|
|       1|         237|         224|         1|
|       1|         170|         229|         1|
|       1|         236|         236|         1|
|       1|         262|         140|         1|
|       1|         141|         237|         1|
|       1|         170|         224|         1|
|       1|         229|         237|         1|
|       1|         143|         143|         1|
|       1|         239|         163|         1|
|       1|          90|         223|         1|
|       1|         166|         237|         1|
|       1|          68|         140|         1|
|       1|         116|          65|         1|
|       1|         162|         161|         1|
|       1|         142|         143|         1|
|       1|         142|          82|    